<a href="https://colab.research.google.com/github/spaudel/Heat-AI/blob/main/HeatAI_temperature_anomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Standard libraries
import os
import math
import numpy as np
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()

## Pandas for reading and processing csvs
import pandas as pd

## Json decode
import json

## Use requests to get api data
import requests

## Progress bar
from tqdm.notebook import tqdm

In [ ]:

district_id = '64' #surkhet. TODO: create a function for this
from_date_time = '2024-09-25'
to_date_time = '2024-10-05'

# the below url gets hourly forecast for the district for the given start and end date
request_url = 'https://heatai.dev.techcolab.org/api/v1/aiapis/district-location-forecasts/?district=64&fromdatetime=2024-09-25&todatetime=2024-10-05'
surkhet_forecast_output = requests.get(request_url)

surkhet_forecast_dict = surkhet_forecast_output.json()

district_id = '66' #banke. TODO: create a function for this


# the below url gets hourly forecast for the district for the given start and end date
request_url = 'https://heatai.dev.techcolab.org/api/v1/aiapis/district-location-forecasts/?district=66&fromdatetime=2024-09-25&todatetime=2024-10-05'
banke_forecast_output = requests.get(request_url)

banke_forecast_dict = banke_forecast_output.json()

district_id = '55' #salyan. TODO: create a function for this


# the below url gets hourly forecast for the district for the given start and end date
request_url = 'https://heatai.dev.techcolab.org/api/v1/aiapis/district-location-forecasts/?district=55&fromdatetime=2024-09-25&todatetime=2024-10-05'
salyan_forecast_output = requests.get(request_url)

salyan_forecast_dict = salyan_forecast_output.json()


district_id = '65' #bardiya. TODO: create a function for this


# the below url gets hourly forecast for the district for the given start and end date
request_url = 'https://heatai.dev.techcolab.org/api/v1/aiapis/district-location-forecasts/?district=65&fromdatetime=2024-09-25&todatetime=2024-10-05'
bardiya_forecast_output = requests.get(request_url)

bardiya_forecast_dict = bardiya_forecast_output.json()

In [ ]:
# Decode the dictionary to create an array
def get_forecast_array(forecast_dict):
  array_dict = {}
  for key in forecast_dict[0].keys():
    for x in range(len(forecast_dict)):
      if key in array_dict:
        array_dict[key].append(forecast_dict[x][key])
      else:
        array_dict[key] = [forecast_dict[x][key]]
  return array_dict;



In [ ]:
surkhet_array_dict = get_forecast_array(surkhet_forecast_dict)
banke_array_dict = get_forecast_array(banke_forecast_dict)
salyan_array_dict = get_forecast_array(salyan_forecast_dict)
bardiya_array_dict = get_forecast_array(bardiya_forecast_dict)

In [ ]:
plt.plot(surkhet_array_dict['air_temperature'])
plt.plot(banke_array_dict['air_temperature'])
plt.plot(salyan_array_dict['air_temperature'])
plt.plot(bardiya_array_dict['air_temperature'])
plt.legend(['surkhet','banke','salyan', 'bardiya'])

In [ ]:
plt.plot(surkhet_array_dict['relative_humidity'])
plt.plot(banke_array_dict['relative_humidity'])
plt.plot(salyan_array_dict['relative_humidity'])
plt.plot(bardiya_array_dict['relative_humidity'])
plt.legend(['surkhet','banke','salyan', 'bardiya'])

In [ ]:
print(surkhet_forecast_dict[0].keys())

In [ ]:
plt.plot(surkhet_array_dict['calculated_heat_index'])
plt.plot(banke_array_dict['calculated_heat_index'])
plt.plot(salyan_array_dict['calculated_heat_index'])
plt.plot(bardiya_array_dict['calculated_heat_index'])
plt.legend(['surkhet','banke','salyan', 'bardiya'])

In [ ]:
def extract_month(x):
     return pd.Series(x.split('-'))

In [ ]:
# Load csv of historical data

test_metdata_url = "https://raw.githubusercontent.com/spaudel/Heat-AI/refs/heads/main/Badhaiyatal_open-meteo-28.15N81.45E146m.csv?token=GHSAT0AAAAAACZR6EWGT5LBO4VBWXNV5JDOZY5JSAQ"
test_df = pd.read_csv(test_metdata_url)

test_dates = test_df['time'].apply(lambda x: extract_month(x))
test_dates.columns = ['yy','mm','dd']

test_df_summer = test_df.loc[test_dates['mm'].isin(['05','06','07','08'])] #hardcoded summer months: May - August. TODO: Make this editable

#Plot summer temperatures
plt.plot(range(len(test_df_summer)),test_df_summer['temperature_2m_max (°C)'])
plt.plot(range(len(test_df_summer)),test_df_summer['temperature_2m_min (°C)'])
plt.plot(range(len(test_df_summer)),test_df_summer['temperature_2m_mean (°C)'])
plt.legend(['max','min','mean'])